# 4. 베이스라인 추천 알고리즘

이 챕터에서는 기본적인 추천 알고리즘을 구현합니다. 다른 고급 알고리즘과의 비교를 위한 기준선 역할을 합니다.

## 4.1 필요한 라이브러리 불러오기

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
from collections import defaultdict
from typing import Dict, List

# 상위 디렉토리 경로를 시스템 경로에 추가하여 utils 모듈을 import할 수 있게 함
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# utils 모듈에서 필요한 클래스와 함수를 import
from utils.models import Dataset, RecommendResult, Metrics
from utils.data_loader import DataLoader

## 4.2 기본 추천 인터페이스

In [1]:
class BaseRecommender:
    def recommend(self, dataset: Dataset, **kwargs) -> RecommendResult:
        """
        추천 알고리즘의 기본 인터페이스.
        모든 추천 모델은 이 메서드를 구현해야 함.
        
        Args:
            dataset: 훈련 및 테스트 데이터를 포함하는 데이터셋
            **kwargs: 추가 매개변수
            
        Returns:
            RecommendResult: 평점 예측 및 추천 아이템 목록
        """
        raise NotImplementedError

NameError: name 'Dataset' is not defined

## 4.3 전체 평균 기반 추천 시스템

In [ ]:
class GlobalMeanRecommender(BaseRecommender):
    """전체 평균 기반 추천 시스템 (Baseline)

    가장 단순한 형태의 추천 시스템으로, 모든 예측값을 전체 평점의 평균으로 설정합니다.
    실제 추천 목록은 생성하지 않으며, 이는 다른 알고리즘의 성능을 비교하는 기준선으로 사용됩니다.

    알고리즘 로직:
    1. 훈련 데이터의 모든 평점 평균(μ)을 계산합니다
    2. 모든 테스트 데이터의 예측값을 이 평균으로 설정합니다
    3. 예측값을 0.5~5.0 범위로 제한합니다
    4. 실제 추천 아이템 목록은 빈 리스트를 반환합니다
    """
    def recommend(self, dataset: Dataset, **kwargs) -> RecommendResult:
        mu = float(dataset.train['rating'].mean())
        pred = pd.Series(mu, index=dataset.test.index).clip(lower=0.5, upper=5.0)
        return RecommendResult(pred, defaultdict(list))

## 4.4 인기도 기반 추천 시스템

In [ ]:
class PopularityRecommender(BaseRecommender):
    """인기도 기반 추천 시스템

    가장 많은 사용자가 평가한(인기 있는) 아이템을 추천하는 방식으로,
    협업 필터링의 가장 단순한 형태입니다. 아이템 평가 횟수를 기준으로 인기도를 측정합니다.

    알고리즘 로직:
    1. RMSE 평가를 위한 평점 예측:
       - 각 영화별 평균 평점을 계산합니다
       - 테스트 데이터의 영화 ID에 맞는 평균 평점을 예측값으로 사용합니다
       - 테스트 데이터에 없는 영화는 전체 평균(μ)으로 설정합니다
       - 예측값을 0.5~5.0 범위로 제한합니다

    2. 추천 목록 생성:
       - 영화별로 평가 횟수(cnt)와 평균 평점(mean)을 계산합니다
       - minimum_num_rating 이상 평가받은 영화만 고려합니다
       - 평가 횟수 기준 내림차순, 동률 시 영화 ID 기준 오름차순으로 정렬합니다
       - 각 사용자가 이미 평가한 영화를 제외한 상위 k개 영화를 추천합니다

    매개변수:
    - k: 추천 목록 크기 (기본값: 10)
    - minimum_num_rating: 최소 평가 횟수 기준 (기본값: 0)
    """
    def recommend(self, dataset: Dataset, k=10, minimum_num_rating=0, **kwargs) -> RecommendResult:
        train, test = dataset.train, dataset.test
        mu = float(train['rating'].mean())

        # RMSE용: 영화별 평균 → 없는 영화 μ → clip
        movie_mean = train.groupby('movieId')['rating'].mean().rename('rating_pred')
        pred = test[['movieId']].merge(movie_mean, on='movieId', how='left')['rating_pred'].fillna(mu)
        pred = pd.Series(pred.values, index=test.index).clip(lower=0.5, upper=5.0)

        # 인기순: cnt desc, tie → movieId asc
        stats = (train.groupby('movieId')['rating'].agg(cnt='count', mean='mean').reset_index())
        stats = stats[stats['cnt'] >= minimum_num_rating]
        stats = stats.sort_values(['cnt','movieId'], ascending=[False, True])
        popular = stats['movieId'].tolist()

        seen = train.groupby('userId')['movieId'].apply(set).to_dict()
        user2items = defaultdict(list)
        for u in test['userId'].unique():
            s = seen.get(u, set())
            rec = [m for m in popular if m not in s][:k]
            user2items[u] = rec

        return RecommendResult(pred, user2items)

## 4.5 베이스라인 추천 알고리즘 평가

In [ ]:
# 데이터셋 로드
loader = DataLoader(num_users=100)
dataset = loader.load()

# 메트릭스 계산기
mc = MetricCalculator()
K = 10  # 추천 목록 크기

# 전체 평균 기반 추천 평가
gm_recommender = GlobalMeanRecommender()
gm_result = gm_recommender.recommend(dataset)
gm_metrics = mc.calc(
    dataset.test['rating'].tolist(),
    gm_result.rating.tolist(),
    dataset.test_user2items,
    gm_result.user2items,
    k=K,
    params={"model": "GlobalMean"}
)
print("GlobalMean:", gm_metrics)

# 인기도 기반 추천 평가
pop_recommender = PopularityRecommender()
pop_result = pop_recommender.recommend(dataset, k=K)
pop_metrics = mc.calc(
    dataset.test['rating'].tolist(),
    pop_result.rating.tolist(),
    dataset.test_user2items,
    pop_result.user2items,
    k=K,
    params={"model": "Popularity"}
)
print("Popularity:", pop_metrics)

## 4.6 베이스라인 모델의 한계

베이스라인 모델은 간단하지만 다음과 같은 한계가 있습니다:

1. **GlobalMeanRecommender**:
   - 모든 사용자와 아이템에 동일한 평점 예측
   - 사용자의 개인 취향이나 아이템의 특성을 고려하지 않음
   - 실제 추천 목록을 생성하지 않음

2. **PopularityRecommender**:
   - 모든 사용자에게 동일한 인기 아이템 추천 (이미 평가한 아이템만 제외)
   - 특이 취향을 가진 사용자에게는 부적절한 추천 제공 가능성
   - 롱테일 아이템(인기는 적지만 일부 사용자에게 매우 관련 있는 아이템)을 추천하지 않음

이러한 한계를 극복하기 위해 다음 챕터에서는 사용자와 아이템 간의 관계를 모델링하는 협업 필터링 알고리즘을 살펴보겠습니다.